In [1]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [2]:
from pyspark.sql import SparkSession
import sparknlp
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import Tokenizer, SentimentDLModel
from sparknlp.pretrained import PretrainedPipeline
import logging
import time

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.pretrained import PretrainedPipeline
spark = SparkSession.builder \
    .appName("Reddit Comment Context Builder") \
    .config("spark.executor.memory", "64g")  \
    .config("spark.driver.memory", "32g")  \
    .config("spark.executor.memoryOverhead", "4096") \
    .config("spark.driver.memoryOverhead", "2048")  \
    .config("spark.driver.maxResultSize", "8g") \
    .config("spark.driver.extraClassPath", "/Volumes/LaCie/wsb_archive/postgresql-42.7.3.jar") \
    .config("spark.driver.extraJavaOptions", "-XX:+UseG1GC") \
    .config("spark.executor.extraJavaOptions", "-XX:+UseG1GC") \
    .config("spark.kryoserializer.buffer.max", "200M")\
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.3.2")\
    .config("spark.sql.shuffle.partitions", "200")\
    .getOrCreate()

:: loading settings :: url = jar:file:/Users/binmingli/spark-3.3.3-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/binmingli/.ivy2/cache
The jars for the packages stored in: /Users/binmingli/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4a422740-e1ef-4afd-a4a3-e777b7a21c0b;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.3.2 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-s3;1.12.500 in central
	found com.amazonaws#aws-java-sdk-kms;1.12.500 in central
	found com.amazonaws#aws-java-sdk-core;1.12.500 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.15 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found software.amazon.ion#ion-java;1.0.2 in central
	found joda-time#joda-time;2.8.1 in central
	found com.amazonaws#jmespath-java;1.12.500 i

24/04/03 19:06:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
# # Load a pre-trained sentiment analysis pipeline

pipeline = PretrainedPipeline('analyze_sentiment', lang='en')

analyze_sentiment download started this may take some time.
Approx size to download 4.8 MB
[ | ]analyze_sentiment download started this may take some time.
Approximate size to download 4.8 MB
[ / ]Download done! Loading the resource.


[ — ]

[OK!]


In [4]:
df = spark.read.parquet("./stock_comments/AAPL_comments")

In [5]:
df.show()

+-------------------+-------------+--------------------+
|       datetime_utc|comment_score|        comment_body|
+-------------------+-------------+--------------------+
|2012-04-11 09:46:43|            2|This is a fantast...|
|2012-04-11 10:12:16|            1|           [deleted]|
|2012-04-11 10:39:08|            2|     INTC is on 4/17|
|2012-04-11 11:02:31|            1|straddle, call, s...|
|2012-04-11 11:47:11|            6|GMCR falls, GOOG ...|
|2012-04-11 12:44:33|            1|CROX 4/26\n\nBZH ...|
|2012-04-11 13:02:56|            1|Shorting GOOG and...|
|2012-04-11 13:16:44|            2|I'm looking at CJ...|
|2012-04-11 13:48:27|            2|GRPN earnings rep...|
|2012-04-11 13:54:48|            1|           [deleted]|
|2012-04-11 13:59:34|            2|I'm long CJES. Se...|
|2012-04-11 14:01:27|            1|Shorting FB for t...|
|2012-04-11 14:02:43|            1|BAC 4/19 betting ...|
|2012-04-11 14:19:43|            1|Shorting AAPL ahe...|
|2012-04-11 14:55:43|          

In [6]:
from pyspark.sql.functions import col

# Apply the sentiment analysis pipeline to the comment_body column
df_renamed = df.withColumnRenamed("comment_body", "text")
result = pipeline.transform(df_renamed)

# Show some columns including sentiment result
result.select("text", "sentiment.result").show(truncate=False)
AAPL_sentiment = result.select(
    col("datetime_utc"),
    col("comment_score"),
    col("text").alias("comment_body"),
    col("sentiment.result").alias("comment_sentiment")
)


+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------+
|text                                                                                                                                                                                                                                                                          |result                                                                          |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------

In [7]:
AAPL_sentiment.show()

+-------------------+-------------+--------------------+--------------------+
|       datetime_utc|comment_score|        comment_body|   comment_sentiment|
+-------------------+-------------+--------------------+--------------------+
|2012-04-11 09:46:43|            2|This is a fantast...|[positive, positi...|
|2012-04-11 10:12:16|            1|           [deleted]|                [na]|
|2012-04-11 10:39:08|            2|     INTC is on 4/17|          [positive]|
|2012-04-11 11:02:31|            1|straddle, call, s...|[negative, positive]|
|2012-04-11 11:47:11|            6|GMCR falls, GOOG ...|[positive, negative]|
|2012-04-11 12:44:33|            1|CROX 4/26\n\nBZH ...|          [positive]|
|2012-04-11 13:02:56|            1|Shorting GOOG and...|[negative, negati...|
|2012-04-11 13:16:44|            2|I'm looking at CJ...|          [negative]|
|2012-04-11 13:48:27|            2|GRPN earnings rep...|[negative, negati...|
|2012-04-11 13:54:48|            1|           [deleted]|        

### The below code should work, but somehow it does not work.

In [8]:
AAPL_sentiment = AAPL_sentiment.cache()

In [9]:
from pyspark.sql.functions import col, size, when, arrays_zip, explode, avg

# 'AAPL_sentiment' is the DataFrame and 'comment_sentiment' contains an array of sentiment strings

# Filter out rows where 'comment_sentiment' contains only 'na'
filtered_df = AAPL_sentiment.filter(size(col("comment_sentiment")) > 0)

# Explode the sentiment array to work with individual sentiments
exploded_df = filtered_df.withColumn("individual_sentiment", explode(col("comment_sentiment")))

# Assign scores to sentiments: +1 for 'positive', -1 for 'negative', and 0 for 'na'
scored_df = exploded_df.withColumn("sentiment_score",
                                   when(col("individual_sentiment") == "positive", 1)
                                   .when(col("individual_sentiment") == "negative", -1)
                                   .otherwise(0))

# Group by original DataFrame identifiers (if necessary) and calculate average sentiment score
# 'datetime_utc' and 'comment_score' serve as a unique identifier for rows in 'AAPL_sentiment'
AAPL_sentiment = scored_df.groupBy("datetime_utc", "comment_score", "comment_body").agg(avg("sentiment_score").alias("sentiment_score"))

AAPL_sentiment = AAPL_sentiment.orderBy("datetime_utc")
AAPL_sentiment.show()

+-------------------+-------------+--------------------+------------------+
|       datetime_utc|comment_score|        comment_body|   sentiment_score|
+-------------------+-------------+--------------------+------------------+
|2012-04-11 09:46:43|            2|This is a fantast...|0.3333333333333333|
|2012-04-11 10:12:16|            1|           [deleted]|               0.0|
|2012-04-11 10:39:08|            2|     INTC is on 4/17|               1.0|
|2012-04-11 11:02:31|            1|straddle, call, s...|               0.0|
|2012-04-11 11:47:11|            6|GMCR falls, GOOG ...|               0.0|
|2012-04-11 12:44:33|            1|CROX 4/26\n\nBZH ...|               1.0|
|2012-04-11 13:02:56|            1|Shorting GOOG and...|              -1.0|
|2012-04-11 13:16:44|            2|I'm looking at CJ...|              -1.0|
|2012-04-11 13:48:27|            2|GRPN earnings rep...|              -1.0|
|2012-04-11 13:54:48|            1|           [deleted]|               0.0|
|2012-04-11 

In [10]:
AAPL_sentiment.write.parquet("./stock_sentiments/AAPL_sentiment")